In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端机器学习：MLOps阶段2：实验：使用Vertex AI分布式训练开始

<table align="left">
      <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_distributed_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_distributed_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_distributed_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的生产中MLOps。本教程涵盖了阶段2：实验：从头开始使用Vertex分布式训练。请注意：Colab和Docker之间存在不兼容性，直到平台解决这个问题，Docker部分可能无法正常工作。

### 目标

在本教程中，您将学习如何在使用`Vertex AI`进行训练时使用`Vertex AI分布式训练`。

本教程使用以下谷歌云ML服务：

- `Vertex AI分布式训练`
- `Vertex AI降维服务器`

执行的步骤包括：

- `镜像策略`：在单个带有多个GPU的虚拟机上训练。
- `多工人镜像策略`：在多个虚拟机上训练，并自动设置副本。
- `多工人镜像策略`：在多个虚拟机上训练，并有副本的精细控制。
- `降维服务器`：在多个虚拟机上训练，通过`Vertex AI降维服务器`在虚拟机之间同步更新。
- `TPU训练`：使用多个Cloud TPU进行训练。

### 推荐

在 Google Cloud 上进行端对端 MLOps 时，以下是在何时使用 Vertex AI 分布式训练的最佳实践：

**单个 VM / 单个设备（OneDeviceStrategy）**

您正在进行实验，总的训练数据量和模型参数数量都很少。

如果模型参数数量非常小，则您可能无法从 GPU 中获得太大的好处，可以考虑使用 VM 的 CPU。

**单个 VM / 多个计算设备（MirroredStrategy）**

模型参数数量非常大，但总的训练数据量较少。

**多个 VM / 多个计算设备（MultiWorkerMirroredStrategy）**

模型参数数量非常大，总的训练数据量也非常大。

**ReductionServer**

在大量 VM 之间进行训练时，需要同步的模型参数更新量非常大。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已内置于 TensorFlow 中。训练后的模型预测房屋的中位价格，单位为千美元。

### 成本

本教程使用 Google Cloud 的计费组件：

Vertex AI
Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)，根据您的预期使用量生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform -q

重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的Google Cloud项目

**无论您使用哪种笔记本环境，都需要完成以下步骤。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您可以获得300美元的免费信用额，用于支付计算/存储成本。

1. [确保您的项目已启用结算功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、BigQuery、Compute Engine和Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter在以`!`为前缀的行上运行作为shell命令，并且它会将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在Vertex AI上使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能会使用共享测试账户或项目。为了避免在创建的资源之间的用户名称冲突，您需要为每个实例会话创建一个时间戳，并将该时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过验证。请跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务账号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账号**。

3. 在**服务账号名称**字段中输入一个名称，然后点击**创建**。

4. 在**为此服务账号分配访问项目权限**部分，点击**角色**下拉列表。在筛选框中输入“Vertex AI”，并选择**Vertex AI管理员**。在筛选框中输入“存储对象管理员”，并选择**存储对象管理员**。

5. 点击*创建*。包含您的密钥的JSON文件将下载到本地环境中。

6. 在下面的单元格中将您的服务账号密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必需的。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在会话之间保留的地方。

在下方设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚未存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容验证对 Cloud Storage 存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

设置变量

接下来，设置一些在教程中使用的变量。
导入库和定义常数

In [ ]:
import google.cloud.aiplatform as aip

### 初始化 Vertex AI SDK 用于 Python

为您的项目和对应的存储桶初始化 Vertex AI SDK 用于 Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU / TRAIN_NGPU`和`DEPLOY_GPU / DEPLOY_NGPU`，以使用支持GPU的容器镜像，并指定分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器镜像，为每个VM分配4个Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80，4)

否则，指定`(None, None)`以在CPU上运行容器镜像。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3以前的版本对GPU支持将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已经修复。这是由在服务函数中生成的静态图操作引起的。如果您在自己的自定义模型中遇到此问题，请使用支持GPU的TF 2.3容器镜像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为训练和预测设置预构建的Docker容器镜像。 

获取最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9 GB内存
 - `vCPUs`：\ [2, 4, 8, 16, 32, 64, 96 \] 的数量

*注：以下内容不支持用于训练：*

 - `标准`：2个vCPUs
 - `高CPU`：2、4和8个vCPUs

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## 镜像策略

在单个VM上进行训练时，可以选择使用单个计算设备进行训练，也可以在同一个VM上使用多个计算设备进行训练。使用Vertex AI Distributed Training，您可以指定VM实例的计算设备数量和类型：CPU，GPU。

Vertex AI Distributed Training支持TensorFlow模型的`tf.distribute.MirroredStrategy'。要在同一台VM上跨多个计算设备进行训练，您需要在Python训练脚本中执行以下额外步骤：

1. 设置`tf.distribute.MirrorStrategy`
2. 在`tf.distribute.MirrorStrategy`范围内编译模型。*注意：*告知MirroredStrategy在计算设备之间镜像哪些变量。
3. 将每个计算设备的批量大小增加到num_devices * batch size。

在过渡期间，批次的分配以及对模型参数的更新将同步进行。

### 创建并运行自定义训练任务

要训练一个自定义模型，需要执行两个步骤：1）创建一个自定义训练任务，和2）运行该任务。

#### 创建自定义训练任务

可以使用`CustomTrainingJob`类来创建一个自定义训练任务，具有以下参数：

- `display_name`：自定义训练任务的人类可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tar包的形式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：*没有`requirements`参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_boston.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 检查培训包

#### 包布局

在开始培训之前，您将查看Python包如何被组装为一个自定义培训任务。解压后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境的指令。

文件`trainer/task.py`是执行自定义培训任务的Python脚本。*注意*，当我们在工作池规范中引用时，我们将目录斜杠替换为点 (`trainer.task`) 并省略文件后缀（`.py`）。

#### 包装配

在以下小节中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing cloud\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细说明，这只是供您查看的。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 用由 `args.epochs` 指定的时期训练模型（`fit()`）。
- 将经过训练的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirrored and MultiWorker Distributed Training

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
import logging

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=16, type=int,
                    help='Size of a batch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

logging.info('DEVICES'  + str(device_lib.list_local_devices()))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
    logging.info("Single device training")
# Single Machine, multiple compute device
elif args.distribute == 'mirrored':
    strategy = tf.distribute.MirroredStrategy()
    logging.info("Mirrored Strategy distributed training")
# Multi Machine, multiple compute device
elif args.distribute == 'multiworker':
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    logging.info("Multi-worker Strategy distributed training")
    logging.info('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
    # Single Machine, multiple TPU devices
elif args.distribute == 'tpu':
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    tf.config.experimental_connect_to_cluster(cluster_resolver)
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    strategy = tf.distribute.TPUStrategy(cluster_resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))

logging.info('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

def _is_chief(task_type, task_id):
    ''' Check for primary if multiworker training
    '''
    return (task_type == 'chief') or (task_type == 'worker' and task_id == 0) or task_type is None


def get_data():
    # Scaling Boston Housing data features
    def scale(feature):
        max = np.max(feature)
        feature = (feature / max).astype(np.float)
        return feature, max

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
        path="boston_housing.npz", test_split=0.2, seed=113
    )

    params = []
    for _ in range(13):
        x_train[_], max = scale(x_train[_])
        x_test[_], _ = scale(x_test[_])
    params.append(max)

    # store the normalization (max) value for each feature
    with tf.io.gfile.GFile(args.param_file, 'w') as f:
        f.write(str(params))
    return (x_train, y_train), (x_test, y_test)

def get_model():
    model = tf.keras.Sequential([
          tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(1, activation='linear')
    ])

    model.compile(
          loss='mse',
          optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr)
    )
    return model

def train(model, x_train, y_train):
    NUM_WORKERS = strategy.num_replicas_in_sync
    # Here the batch size scales up by number of workers since
    # `tf.data.Dataset.batch` expects the global batch size.
    GLOBAL_BATCH_SIZE = args.batch_size * NUM_WORKERS

    model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)

    if args.distribute == 'multiworker':
        task_type, task_id = (strategy.cluster_resolver.task_type,
                              strategy.cluster_resolver.task_id)
    else:
        task_type, task_id = None, None

    if args.distribute=="tpu":
        save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
        model.save(args.model_dir, options=save_locally)
    # single, mirrored or primary for multiworker
    elif _is_chief(task_type, task_id):
        model.save(args.model_dir)
    # non-primary workers for multi-workers
    else:
        # each worker saves their model instance to a unique temp location
        worker_dir = args.model_dir + '/workertemp_' + str(task_id)
        tf.io.gfile.makedirs(worker_dir)
        model.save(worker_dir)

with strategy.scope():
    # Creation of dataset, and model building/compiling need to be within
    # `strategy.scope()`.
    model = get_model()

(x_train, y_train), (x_test, y_test) = get_data()

train(model, x_train, y_train)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

运行自定义的Python包培训工作

接下来，您运行自定义作业，通过调用run()方法开始训练作业。参数与运行CustomTrainingJob时相同。

In [ ]:
MODEL_DIR = BUCKET_URI

CMDARGS = ["--epochs=5", "--batch_size=16", "--distribute=mirrored"]

model = job.run(
    model_display_name="boston_" + TIMESTAMP,
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
    base_output_dir=MODEL_DIR,
    sync=True,
)

删除自定义训练作业

训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

删除模型

该方法'delete（）'将删除该模型。

In [ ]:
model.delete()

## 多工作进程镜像策略

通过 Vertex AI 分布式训练，您可以使用多个 VM 实例进行训练。

Vertex AI 分布式训练支持 TensorFlow 和 PyTorch 模型的 `tf.distribute.MultiWorkerMirroredStrategy`。要在多个 VM 上进行训练，您需要在您的 Python 训练脚本中执行以下附加步骤：

1. 所有镜像策略的额外步骤，只是将 MirroredStrategy 替换为 MultiWorkerStrategy。
2. 设置工作进程池。
3. 更改模型保存方式，使非主要工作进程将其模型实例保存到一个独特的临时目录中。

*注意：* 您不需要构造 TF_CONFIG 环境变量。它会被 Vertex AI 分布式训练自动构造。

了解有关[分布式训练](https://cloud.google.com/vertex-ai/docs/training/distributed-training)的更多信息。

工作池

如果您在Vertex AI上运行一个分布式训练作业，您需要在训练集群中指定多台机器（节点）。训练服务会为您指定的机器类型分配资源。在给定节点上运行的作业称为副本。具有相同配置的一组副本称为工作池。

训练集群中的每个副本都会被分配一个单一的角色或任务进行分布式训练。例如：

- 主要副本：有一个副本被指定为主要副本。此任务管理其他副本并报告整个作业的状态。

- 工作者：可以指定一个或多个副本为工作者。这些副本根据您在作业配置中指定的工作量进行工作。

- 参数服务器：如果您的ML框架支持，可以指定一个或多个副本为参数服务器。这些副本存储模型参数并协调工作者之间共享模型状态。

- 评估器：如果您的ML框架支持，可以指定一个或多个副本作为评估器。这些副本可用于评估您的模型。如果您正在使用TensorFlow，请注意，TensorFlow通常希望您使用不超过一个评估器。

要配置分布式训练作业，请定义工作池列表（workerPoolSpecs[]），为每种任务指定一个WorkerPoolSpec：

*注意：*工作池是有顺序要求的（0..3）：

**workerPoolSpecs[0]**：主要，负责人，调度器，或“主节点”

**workerPoolSpecs[1]**：次要，副本，工作者

**workerPoolSpecs[2]**：参数服务器，减少服务器

**workerPoolSpecs[3]**：评估器

### 多工镜像策略的分布式培训选项

设置工作人员池取决于您用于培训的Vertex AI方法。

**CustomTrainingJob** / **CustomContainerTrainingJob** / **CustomPythonPackageTrainingJob**

`replica_count` 包括主要和次要（replica_count-1），并共享相同的机器类型和加速器。

您不能指定参数服务器或评估节点。

**CustomJob**

您可以指定一个 `worker_pool_spec`，在其中您可以为每个四个工作者池指定详细设置。

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要进行两个步骤：1）创建一个自定义训练作业，以及2）运行该作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball格式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：* 没有`requirements`参数。您可以在Python包中的`setup.py`脚本中指定任何依赖项。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_boston.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

运行自定义Python包训练作业

接下来，您可以运行自定义作业，通过调用`run()`方法来启动训练作业。参数与运行CustomTrainingJob时相同。

In [ ]:
MODEL_DIR = BUCKET_URI

CMDARGS = ["--epochs=5", "--batch_size=16", "--distribute=multiworker"]

try:
    model = job.run(
        model_display_name="boston_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=4,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
except Exception as e:
    # may fail duing model.save() -- seems to be some issue when merging checkpoints from the workers
    print(e)

删除自定义训练作业

训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，训练作业可以使用`cancel()`方法取消。

In [ ]:
job.delete()

### 使用CustomJob进行多工作节点的分布式训练

使用`CustomJob`进行多工作节点的分布式训练具有对主副本的精细控制以及可选择为参数服务器和评估器指定工作池的优势。创建一个`CustomJob`包括以下步骤：

1. 为每个工作池指定个别细节。
2. 将训练包嵌入Docker镜像中。

创建一个Docker文件

为了使用您自己定制的训练容器，您需要构建一个Docker文件，并将您的训练脚本嵌入到容器中。

#### 编写Docker文件内容

将您的代码放入容器的第一步是创建一个Docker文件。在您的Docker文件中，您将包含运行容器镜像所需的所有命令。它将安装您正在使用的所有库并设置培训代码的入口点。

1. 从TensorFlow存储库中安装一个预定义的深度学习图像容器镜像。
2. 复制Python培训代码，稍后将显示。
3. 将入口点设置为Python培训脚本`trainer/task.py`。请注意，ENTRYPOINT命令中省略了`.py`，因为它被暗示了。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 本地构建容器

接下来，您将为您的客户容器提供一个名称，以便在提交到 Google Container Registry 时使用。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/boston:v1"

接下来，建造容器。

In [ ]:
if not IS_COLAB:
    ! docker build custom -t $TRAIN_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

#### 本地测试容器

在您的笔记本实例中运行容器，以确保其正常工作。您将运行5个时代。

In [ ]:
if not IS_COLAB:
    ! docker run $TRAIN_IMAGE --epochs=5 --model-dir=./

#### 注册自定义容器

当您在本地运行完容器后，请将其推送到 Google 容器注册表。

In [ ]:
if not IS_COLAB:
    ! docker push $TRAIN_IMAGE

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $TRAIN_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build custom -t $2
docker run $2 --epochs=5 --model-dir=./
docker push $2
kill $(jobs -p)

主要工作池

主要工作池（索引0）协调所有其他副本完成的工作。将replicaCount设为1。由于工作者是协调而不是训练，所以请使用通用用途的CPU，而不是GPU。

了解更多关于[训练的机器类型](https://cloud.google.com/vertex-ai/docs/training/configure-compute#machine-types)。

In [ ]:
PRIMARY_COMPUTE = "n2-highcpu-64"

MODEL_DIR = BUCKET_URI

CMDARGS = [
    "--model-dir=" + MODEL_DIR,
    "--epochs=5",
    "--batch_size=16",
    "--distribute=multiworker",
]

CONTAINER_SPEC = {"image_uri": TRAIN_IMAGE, "command": "trainer.task", "args": CMDARGS}

PRIMARY_WORKER_POOL = {
    "replica_count": 1,
    "machine_spec": {"machine_type": PRIMARY_COMPUTE, "accelerator_count": 0},
    "container_spec": CONTAINER_SPEC,
}

WORKER_POOL_SPECS = [PRIMARY_WORKER_POOL]

培训工人池

次要的工人池（索引1）执行模型训练。每个副本都会安装一个您的训练包的实例。

每个副本可能具有一个（单设备训练）或多个（镜像）用于训练的计算设备。

In [ ]:
TRAIN_WORKER_POOL = {
    "replica_count": 4,
    "machine_spec": {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_count": TRAIN_NGPU,
        "accelerator_type": TRAIN_GPU,
    },
    "container_spec": CONTAINER_SPEC,
}

WORKER_POOL_SPECS.append(TRAIN_WORKER_POOL)

### 使用工作池规格创建CustomJob

接下来，您可以为多工作人员分布式培训工作创建一个`CustomJob`：

- `display_name`：自定义工作的显示名称。

- `worker_pool_specs`：每个工作池的详细规格。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aip.CustomJob(display_name=DISPLAY_NAME, worker_pool_specs=WORKER_POOL_SPECS)

运行自定义任务

接下来，运行自定义任务。

In [ ]:
try:
    job.run(sync=True)
except Exception as e:
    # may fail in multi-worker to find startup script
    print(e)

### 删除自定义训练作业

在训练作业完成后，您可以使用`delete()`方法来删除训练作业。 在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

减少服务器

为了加快大型模型训练的速度，许多工程团队正在采用分布式训练，利用ML加速器的规模扩展集群。然而，规模化的分布式训练也带来了一系列挑战。具体来说，节点之间的有限网络带宽使得优化分布式训练的性能困难重重，特别是对于大型集群配置。

Vertex AI减少服务器优化了NVIDIA GPU上的多节点分布式训练的带宽和延迟，适用于同步数据并行算法。同步数据并行是许多广泛采用的分布式训练框架的基础，包括TensorFlow的MultiWorkerMirroredStrategy、Horovod和PyTorch Distributed。通过优化这些框架使用的全局约减集合操作的带宽使用和延迟，减少服务器可以降低大型训练作业的时间和成本。

了解更多关于[使用Vertex Reduction Server优化训练性能](https://cloud.google.com/blog/topics/developers-practitioners/optimize-training-performance-reduction-server-vertex-ai)。

In [ ]:
reduction_server_count = 1
reduction_server_machine_type = "n1-highcpu-16"
reduction_server_image_uri = (
    "us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest"
)

PARAMETER_POOL = {
    "replica_count": reduction_server_count,
    "machine_spec": {
        "machine_type": reduction_server_machine_type,
    },
    "container_spec": {"image_uri": reduction_server_image_uri},
}
WORKER_POOL_SPECS.append(PARAMETER_POOL)

### 使用工作池规范创建CustomJob

接下来，您可以为多工作程序分布式训练作业创建一个`CustomJob`：

- `display_name`：自定义作业的显示名称。

- `worker_pool_specs`：每个工作池的详细规范。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aip.CustomJob(display_name=DISPLAY_NAME, worker_pool_specs=WORKER_POOL_SPECS)

运行自定义作业

接下来，运行自定义作业。

In [ ]:
try:
    job.run(sync=True)
except Exception as e:
    # may fail in multi-worker to find startup script
    print(e)

### 删除自定义训练作业

在训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

## 云 TPU 训练

为了进一步加快训练速度，您的组织可以利用谷歌的云张量处理单元（TPU）架构。

云 TPU 是定制设计的机器学习 ASIC，用于支持谷歌产品如翻译、照片、搜索、助手和Gmail。云 TPU 旨在在谷歌云上运行最先进的机器学习模型和人工智能服务。其定制的高速网络在单个架构中提供超过 100 petaflops 的性能。

了解更多关于[云 TPU](https://cloud.google.com/tpu)

*注意*：TPU VM 训练目前是一项自愿功能。您的 GCP 项目必须首先被添加到功能白名单中。请将您的项目信息（项目 ID/编号）发送至 vertex-ai-tpu-vm-training-support@google.com 进行白名单处理。一旦您的项目准备就绪，您将会收到一封电子邮件通知。

### 为TPU训练编写Docker文件

目前，还没有预构建的Vertex AI Docker镜像用于与TPUs训练。没关系，您可以自己创建，具体步骤如下：

1. 创建一个原始的 Python 3 镜像（例如，`python3:8`）。
2. 获取并安装TPU库（`libtpu.so`）。
3. 复制您的训练程序进入。

In [ ]:
%%writefile custom/Dockerfile
FROM python:3.8

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

RUN pip3 install tensorflow-datasets

# Install TPU Tensorflow and dependencies.
# libtpu.so must be under the '/lib' directory.
RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/libtpu/20210525/libtpu.so -O /lib/libtpu.so
RUN chmod 777 /lib/libtpu.so

RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/20210525/tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN pip3 install tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN rm tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建并推送Docker镜像至存储库。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/tpu-train:latest"

os.chdir("custom")
! docker build --quiet --tag={TRAIN_IMAGE} .
! docker push {TRAIN_IMAGE}
os.chdir("..")

TPU 工作者规范池

接下来，创建工作者规范池。对于 TPU，您需要：

- 创建仅一个工作者池（主要）。
- 将机器类型设置为 `cloud-tpu`。
- 将加速器类型设置为 `TPU`。

In [ ]:
# Use TPU Accelerators. Temporarily using numeric codes, until types are added to the SDK
#   6 = TPU_V2
#   7 = TPU_V3
TRAIN_TPU, TRAIN_NTPU = (7, 8)
TRAIN_COMPUTE = "cloud-tpu"


if not TRAIN_NTPU or TRAIN_NTPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "tpu"
print(TRAIN_STRATEGY)

EPOCHS = 20
STEPS = 10000

TRAINER_ARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
]


WORKER_POOL_SPECS = [
    {
        "container_spec": {
            "args": TRAINER_ARGS,
            "image_uri": TRAIN_IMAGE,
        },
        "replica_count": 1,
        "machine_spec": {
            "machine_type": TRAIN_COMPUTE,
            "accelerator_type": TRAIN_TPU,
            "accelerator_count": TRAIN_NTPU,
        },
    }
]

print(WORKER_POOL_SPECS[0])

### 使用worker池规格创建CustomJob

接下来，您将为多工作站分布式训练任务创建一个`CustomJob`：

- `display_name`: 自定义工作的显示名称。
- `worker_pool_specs`: 每个工作站池的详细规格。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aip.CustomJob(display_name=DISPLAY_NAME, worker_pool_specs=WORKER_POOL_SPECS)

运行自定义作业

接下来，运行自定义作业。

In [ ]:
try:
    job.run(sync=True)
except Exception as e:
    # may fail in multi-worker to find startup script
    print(e)

删除自定义训练作业

训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 云存储存储桶

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI